In [24]:
import pandas as pd
import numpy as np
data = {
    "Aspecto": ["Sol", "Sol", "Nuvens", "Chuva", "Chuva", "Chuva", "Nuvens", "Sol", "Sol", "Chuva", "Sol", "Nuvens", "Nuvens", "Chuva"],
    "Temp": ["Quente", "Quente", "Quente", "Ameno", "Fresco", "Fresco", "Fresco", "Ameno", "Fresco", "Ameno", "Ameno", "Quente", "Quente", "Ameno"],
    "Humidade": ["Elevada", "Elevada", "Elevada", "Elevada", "Normal", "Normal", "Normal", "Normal", "Normal", "Elevada", "Normal", "Elevada", "Normal", "Elevada"],
    "Vento": ["Fraco", "Forte", "Fraco", "Fraco", "Fraco", "Forte", "Fraco", "Forte", "Fraco", "Fraco", "Forte", "Fraco", "Forte", "Forte"],
    "Tenis": ["Não", "Não", "Sim", "Sim", "Sim", "Não", "Sim", "Não", "Sim", "Sim", "Sim", "Sim", "Sim", "Não"]
}
df = pd.DataFrame(data)
display(df)

,Aspecto,Temp,Humidade,Vento,Tenis
0,Sol,Quente,Elevada,Fraco,Não
1,Sol,Quente,Elevada,Forte,Não
2,Nuvens,Quente,Elevada,Fraco,Sim
3,Chuva,Ameno,Elevada,Fraco,Sim
4,Chuva,Fresco,Normal,Fraco,Sim
5,Chuva,Fresco,Normal,Forte,Não
6,Nuvens,Fresco,Normal,Fraco,Sim
7,Sol,Ameno,Normal,Forte,Não
8,Sol,Fresco,Normal,Fraco,Sim
9,Chuva,Ameno,Elevada,Fraco,Sim


In [25]:
def entropia(col):
    counts = np.unique(col, return_counts=True)
    N = float(col.shape[0])
    ent = 0.0
    for ix in counts[1]:
        p = ix / N
        ent += -1.0 * p * np.log2(p)
    return ent

In [26]:

def information_gain(df, attr, target):
    total = entropia(df[target])
    valores = np.unique(df[attr])
    acc = 0
    for v in valores:
        subset = df[df[attr] == v][target]
        x = len(subset) / len(df)
        acc += x * entropia(subset)
    return total - acc

### Explicação da Função `information_gain`

A função `information_gain(df, attr, target)` calcula o ganho de informação ao dividir os dados do DataFrame `df` pelo atributo `attr` para prever o atributo alvo `target`. Veja como ela funciona passo a passo:

- **`total`**:  
  Calcula a entropia do atributo alvo (`target`).  
  Exemplo: se `target` for `"Tenis"`, estamos medindo a desordem (incerteza) na coluna `'Tenis'`, ou seja, o quão misturado está o "Sim"/"Não".

- **`valores`**:  
  Guarda os valores únicos do atributo escolhido (`attr`).  
  Exemplo: se o atributo for `"Aspecto"`, `valores` será `["Sol", "Nuvens", "Chuva"]`.

- **Laço `for v in valores`:**  
  Para cada valor possível do atributo, por exemplo, `v = "Sol"`, `v = "Nuvens"` ou `v = "Chuva"`:
    - **`subset`**:  
      Filtra o DataFrame, pegando só as linhas onde o atributo (`attr`) é igual a `v`.  
      Depois, seleciona apenas a coluna do target (`Tenis`).
    - **`x = len(subset) / len(df)`**:  
      Calcula a proporção de linhas do DataFrame que têm aquele valor do atributo.
    - **`acc += x * entropia(subset)`**:  
      Soma (acumula) a entropia do subset, multiplicada pela proporção daquele valor no DataFrame.

- **`return total - acc`**:  
  O ganho de informação é a diferença entre a entropia total e a soma das entropias ponderadas dos subconjuntos.  
  Isso mostra quanto saber o atributo (`attr`) reduz a incerteza sobre o target (`Tenis`).

  Se quisermos calcular o ganho de informação ao dividir pelo atributo `"Aspecto"` para prever `"Tenis"`, a função irá:

1. Calcular a entropia de `"Tenis"` para todas as linhas do DataFrame.
2. Para cada valor de `"Aspecto"` (`"Sol"`, `"Nuvens"`, `"Chuva"`), calcular a entropia de `"Tenis"` apenas nas linhas daquele aspecto e ponderar pelo número de ocorrências desse aspecto.
3. Retornar a diferença entre a entropia total e essa soma ponderada.

Assim, descobrimos **quanto saber o "Aspecto" do tempo ajuda a prever se alguém vai jogar tênis ou não**.

In [27]:
class DecisionTreeCategorical:
    def __init__(self, depth=0, max_depth=3):
        self.children = {}
        self.attr = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None

    def train(self, df, features, target):
        if len(np.unique(df[target])) == 1 or len(features) == 0 or self.depth >= self.max_depth:
            self.target = df[target].mode()[0]
            return

        gains = [information_gain(df, attr, target) for attr in features]
        best_attr = features[np.argmax(gains)]
        self.attr = best_attr
        self.children = {}

        for v in np.unique(df[best_attr]):
            subset = df[df[best_attr] == v]
            if subset.empty:
                self.children[v] = None
            else:
                child = DecisionTreeCategorical(depth=self.depth+1, max_depth=self.max_depth) #Aqui definimos Child como objeto
                child.train(subset, [f for f in features if f != best_attr], target)
                self.children[v] = child #Para o valor v do atributo, o filho correspondente é o objeto child.

    def predict(self, row):
        if self.attr is None or self.children == {}:
            return self.target
        val = row[self.attr]
        if val in self.children and self.children[val] is not None:
            return self.children[val].predict(row)
        else:
            return self.target  

## Explicação do código: Classe `DecisionTree`

### 1. Inicialização da Árvore (`__init__`)

Ao criar uma instância da árvore de decisão, o método `__init__` é chamado para inicializar seus atributos:

- **`self.children = {}`**  
  - Um dicionário vazio que irá armazenar os nós filhos da árvore.  
  - Cada chave corresponde a um valor do atributo de decisão (por exemplo, "Sol", "Chuva", etc.), e cada valor é uma subárvore (outro objeto `DecisionTree`).

- **`self.attr`**  
  - Este atributo armazenará o nome do atributo escolhido para dividir os dados naquele nó específico da árvore.  
  - Cada nó da árvore pode ter um `self.attr` diferente, dependendo do melhor atributo selecionado para o split naquele ponto.

- **`self.max_depth` & `self.depth`**  
  - `self.max_depth`: Define a profundidade máxima permitida para a árvore.  
  - `self.depth`: Indica em que nível da árvore o nó atual está (a raiz começa em 0).

- **`self.target`**  
  - Este atributo será preenchido em nós folha com o valor mais comum (modo) da variável alvo (`target`) nos dados daquele nó.
---
## 2.Critérios de Parada na Construção da Árvore de Decisão

Durante o treinamento da árvore de decisão (método `train`), avaliamos algumas condições para determinar se devemos parar a criação de novos nós e transformar o nó atual em uma **folha**. Os principais critérios de parada são:

1. **Limite de Profundidade**  
   - Checamos se `self.depth >= self.max_depth`, ou seja, se a árvore já atingiu o limite máximo de profundidade permitido.

2. **Target Puro**  
   - Verificamos se todos os valores do nosso alvo (`target`) são iguais, como por exemplo `[yes, yes, yes]`. Nesse caso, não faz sentido continuar dividindo, pois todos os exemplos possuem o mesmo rótulo.

3. **Acabaram as Features**  
   - Conferimos se ainda existem atributos (`features`) disponíveis para realizar divisões. Se a lista estiver vazia, não é possível continuar.



Quando qualquer uma dessas condições for satisfeita, o nó atual se torna uma folha, e atribuimos a ele o valor **mais comum** do nosso target no subconjunto de dados analisado.  
Por exemplo, se o target no nó é `[yes, yes, yes]`, o valor atribuído será `yes`.

---
## 3.Escolha da Melhor Feature para Divisão

Na construção da árvore de decisão, seguimos os passos abaixo para selecionar o atributo que mais contribui para a separação dos dados:

1. **Cálculo do Ganho de Informação**  
   - Calculamos o ganho de informação para cada uma das features disponíveis e armazenamos esses valores em uma lista.

2. **Identificação do Maior Ganho**  
   - Usamos `np.argmax(gains)` para encontrar o índice do maior valor de ganho de informação na lista.

3. **Seleção da Melhor Feature**  
   - Selecionamos a feature correspondente a esse índice e armazenamos seu nome na variável `best_attr`.  
   - Esta será a feature utilizada para dividir o nó atual da árvore.
## 4. Como funciona a recursividade na construção dos filhos da Decision Tree?

No método `train` da nossa árvore de decisão, após escolher o melhor atributo (`best_attr`) para dividir o nó atual, seguimos os seguintes passos para construir seus filhos:

---

1. **Loop sobre os valores únicos do atributo selecionado**  
   Utilizamos um `for` para percorrer cada valor possível de `best_attr`.  
   **Exemplo:**  
   Se `best_attr` for `"Aspecto"` e os valores possíveis forem `[Sol, Nuvens, Chuva]`, o loop irá funcionar assim:
   - Primeira iteração: `x = "Sol"`
   - Segunda iteração: `x = "Nuvens"`
   - Terceira iteração: `x = "Chuva"`

2. **Filtragem do DataFrame para cada valor**  
   Para cada valor `x`, criamos um subconjunto do DataFrame apenas com as linhas onde `best_attr` é igual a `x`.  
   **Exemplo:**  
   - Para `x = "Sol"`, `sub` será todas as linhas onde `"Aspecto" == "Sol"`.
   - Para `x = "Nuvens"`, `sub` será todas as linhas onde `"Aspecto" == "Nuvens"`.
   - Para `x = "Chuva"`, `sub` será todas as linhas onde `"Aspecto" == "Chuva"`.

3. **Criação do nó filho ("Child")**  
   Para cada subconjunto `sub`, instanciamos um novo objeto `DecisionTree`, que será o nó filho correspondente ao valor `x`.  
   **Exemplo:**  
   - O nó `"Sol"` será um filho do nó atual, responsável por processar apenas os exemplos com `"Aspecto" == "Sol"`.

4. **Recursividade para treinar o filho**  
   Chamamos `child.train(sub, list(filter(lambda f: f != best_attr, features)), target)`.  
   O que acontece aqui?  
   - Chamamos novamente a função `train`, agora para o filho.  
   - O DataFrame de entrada (`sub`) contém apenas exemplos onde o atributo tem valor `x` (por exemplo, apenas `"Sol"`).
   - A lista de features é atualizada retirando `best_attr`, pois já foi usada para a divisão.
   - O processo se repete: o filho tentará encontrar o melhor atributo para dividir os dados do seu subconjunto.

   **Exemplo prático:**  
   Imagine que no nó atual, `best_attr` é `"Aspecto"` e estamos na iteração `x = "Sol"`.  
   - O DataFrame `sub` contém apenas linhas com `"Aspecto" == "Sol"`.
   - Criamos o nó filho `"Sol"`.
   - Chamamos `train` para esse filho, que irá procurar agora o melhor atributo (exceto `"Aspecto"`, que já foi usado) para dividir ainda mais o subconjunto de dados
### Resumindo

A cada iteração, dividimos o DataFrame conforme os valores do melhor atributo, criamos um nó filho para cada valor, e chamamos recursivamente o método `train` para cada filho, processando apenas o subconjunto de dados correspondente.  
Dessa forma, a árvore vai “crescendo” de cima para baixo, especializando cada nó conforme os dados vão sendo filtrados!

---

## (Importante) Explicação da função `predict`

A função `predict` é responsável por prever o valor alvo (classe) para uma nova entrada na árvore de decisão. Vamos entender seu funcionamento passo a passo:

---

### 1) Entrada da função

- O parâmetro de entrada é `row`, que representa **uma linha do DataFrame** (ou seja, um exemplo com todos os atributos preenchidos).
- **Exemplo:**  
  ```python
  row = {
      "Aspecto": "Sol",
      "Temp": "Quente",
      "Humidade": "Elevada",
      "Vento": "Fraco"
  }
  ```

---

### 2) Verificação de nó folha

- O código verifica:
  ```python
  if self.attr is None or self.children == {}:
      return self.target
  ```
- Isso significa que **se o nó atual não tem mais atributo para dividir (`self.attr is None`) ou não tem filhos (`self.children == {}`), então é uma folha**.
- Nesse caso, a função retorna o valor de `self.target`, que é o valor mais comum do target naquele nó.
- **Exemplo:**  
  Se o nó representa todas as situações com `"Aspecto" == "Sol"` e a maioria delas resulta em `"Tenis" == "Não"`, então `self.target = "Não"`.

---

### 3) Definindo o valor do atributo de decisão

- A linha:
  ```python
  val = row[self.attr]
  ```
- **Significa:**  
  Pegue o valor do atributo que está sendo usado para dividir neste nó.
- **Exemplo:**  
  Se `self.attr == "Aspecto"` e `row["Aspecto"] == "Sol"`, então `val = "Sol"`.

---

### 4) Decidindo para qual filho ir

- O código verifica:
  ```python
  if val in self.children and self.children[val] is not None:
      return self.children[val].predict(row)
  else:
      return self.target
  ```
- **Ou seja:**
  - Se existe um filho correspondente ao valor de `val` (por exemplo, "Sol") e esse filho não é nulo, chamamos recursivamente a função `predict` no filho.
  - **Exemplo Prático:**  
    - Se estamos em um nó que divide por `Aspecto`, e `row["Aspecto"] == "Sol"`, então procuramos o filho `self.children["Sol"]`.
    - Se existir, chamamos `predict` nesse filho, passando a mesma linha.
    - Se esse filho for uma folha (não tem mais atributos para dividir), ele retorna seu `self.target` (por exemplo, "Sim" ou "Não").

- **Se não existir filho para esse valor**, retornamos o valor alvo mais comum do nó atual (`self.target`).

---

### **Fluxo resumido com exemplo**

1. **Começamos na raiz:**  
   - `self.attr == "Aspecto"`
   - `row["Aspecto"] == "Sol"`
   - Vamos para o filho `"Sol"`

2. **Filho "Sol":**
   - Suponha que não há mais atributos para dividir (folha)
   - `self.target == "Não"`
   - Retorna `"Não"` como predição

---

### **Resumo**

- A cada chamada, a função verifica se é folha; se não for, escolhe o próximo nó filho de acordo com o valor do atributo de decisão.
- O processo é recursivo e termina em uma folha, retornando a classe prevista.

In [28]:

features = ["Aspecto", "Temp", "Humidade", "Vento"]
target = "Tenis"
tree = DecisionTreeCategorical(max_depth=3)
tree.train(df, features, target)


In [29]:
for i in range(len(df)):
    test_row = df.iloc[i]
    print(f"Exemplo {i+1}: Previsto={tree.predict(test_row)}, Real={test_row[target]}")

Exemplo 1: Previsto=Não, Real=Não
Exemplo 2: Previsto=Não, Real=Não
Exemplo 3: Previsto=Sim, Real=Sim
Exemplo 4: Previsto=Sim, Real=Sim
Exemplo 5: Previsto=Sim, Real=Sim
Exemplo 6: Previsto=Não, Real=Não
Exemplo 7: Previsto=Sim, Real=Sim
Exemplo 8: Previsto=Não, Real=Não
Exemplo 9: Previsto=Sim, Real=Sim
Exemplo 10: Previsto=Sim, Real=Sim
Exemplo 11: Previsto=Não, Real=Sim
Exemplo 12: Previsto=Sim, Real=Sim
Exemplo 13: Previsto=Sim, Real=Sim
Exemplo 14: Previsto=Não, Real=Não
